### В данном уроке мы выделим еще несколько потенциально важных фичей из изначального датасета и попробуем применить изученные методы отбора признаков к итоговому датасету.

In [1]:
import numpy as np
import pandas as pd

processed_data = pd.read_csv('processed_data.csv', index_col='id')

processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


In [3]:
initial_data = pd.read_csv('taxi_dataset.csv')

initial_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N


In [4]:
initial_data.shape[0] == processed_data.shape[0]

True

In [5]:
### Вернем в датасет колонку pickup_datetime

initial_data = initial_data.set_index('id')

processed_data = pd.merge(processed_data, initial_data['pickup_datetime'],
                          left_index=True, right_index=True)

In [9]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,pickup_datetime,date,day_of_week,hour,month
id,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,2016-03-14 17:24:55,2016-03-14,0,17,3
id2377394,0,930.399753,0,1.807119,6.498282,2016-06-12 00:43:35,2016-06-12,6,0,6
id3858529,1,930.399753,0,6.392080,7.661527,2016-01-19 11:35:24,2016-01-19,1,11,1
id3504673,1,930.399753,0,1.487155,6.063785,2016-04-06 19:32:31,2016-04-06,2,19,4
id2181028,1,930.399753,0,1.189925,6.077642,2016-03-26 13:30:55,2016-03-26,5,13,3


Напомним, **pickup_datetime** - время начала поездки.

Кажется, что в зависимости от месяца/дня недели/времени суток движение на дорогах может отличаться. Как из-за погодных условий, так и из-за загруженности транспорта. Поэтому, есть подозрение, что будет полезно выделить ряд признаков из колонки **pickup_datetime**. Давайте исследуем зависимость нашей таргетной переменной от указанных факторов.

In [10]:
processed_data['pickup_datetime'] = pd.to_datetime(processed_data['pickup_datetime'])

processed_data['date'] = processed_data.pickup_datetime.dt.date
processed_data['day_of_week'] = processed_data.pickup_datetime.dt.dayofweek
processed_data['hour'] = processed_data.pickup_datetime.dt.hour
processed_data['month'] = processed_data.pickup_datetime.dt.month

In [11]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,pickup_datetime,date,day_of_week,hour,month
id,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,2016-03-14 17:24:55,2016-03-14,0,17,3
id2377394,0,930.399753,0,1.807119,6.498282,2016-06-12 00:43:35,2016-06-12,6,0,6
id3858529,1,930.399753,0,6.392080,7.661527,2016-01-19 11:35:24,2016-01-19,1,11,1
id3504673,1,930.399753,0,1.487155,6.063785,2016-04-06 19:32:31,2016-04-06,2,19,4
id2181028,1,930.399753,0,1.189925,6.077642,2016-03-26 13:30:55,2016-03-26,5,13,3


Исследуем, когда и сколько поездок было совершено. 

Начнем с графиков, показывающих количество поездок в зависимости от времени суток/даты и т.д.

Так же полезно сразу показать и среднее значение таргетной переменной.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()

fig.set_size_inches(16, 10)

ax_1 = fig.add_subplot(1, 2, 1)
plt.hist(processed_data['date'], bins=processed_data.date.unique().shape[0])
plt.ylim((4000, 10000))

ax_2 = fig.add_subplot(1, 2, 2)
plt.bar(sorted(list(processed_data['date'].unique())), 
        processed_data.groupby('date', as_index=False)['log_trip_duration'].mean()['log_trip_duration'],
        width=1)
plt.ylim((6, 7))

fig.tight_layout()

ax_1.set(title = 'Количество поездок по датам')
ax_2.set(title = 'Средняя логарифмированная продолжительность поездок по датам')

plt.show()

Кажется, что внутри недели есть некоторая зависимость таргета от дня недели. 

Такая же зависимость может оказаться и внутри дня (в завимости от часов).

Предлагаю исследовать эту зависимость подробнее.

Изобразите следующие графики зависимостей:

- Количество поездок/Средняя продолжительность поездки vs День недели
- Количество поездок/Средняя продолжительность поездки vs время суток
- Средняя продолжительность поездки vs время суток для каждого дня недели (например, нарисовав с разными цветами и, соответственно, легендами)
- Аналогичные графики, используя информацию о месяце, внутри которого была совершена поездка.
- Ящики с усами для различных: времени суток, дня недели, месяца

Используйте любые доступные инструменты pyplot!

Далее, на основе полученных результатов, мы будем принимать решение о создании новых признаков.

EDA можно использовать не только для того, чтобы понять, какие фичи можно убрать из датасета. Но и для выделения базовых признаков. Этим и займемся!

P.S. Сами графики мы проверять у Вас не будем. Зато для ответа на устные вопросы понадобятся, поэтому, в любом случае, крайне рекомендуем поупражняться. Поэтому рисовать все графики не обязательно - ограничьтесь теми, которые помогут Вам ответить на тестовые вопросы.

**Hint**: обратите внимание на сильную просадку в январе. Почему она могла произойти - можно прочитать <a href="https://en.wikipedia.org/wiki/January_2016_United_States_blizzard"> здесь</a>. В будущем можно будет создать бинарный признак "произошла ли поездка во время сильного снегопада".


In [ ]:
### количество поездок и продолжительность
x = processed_data.groupby('hour').count()['vendor_id']
y = processed_data.groupby('hour').mean()['log_trip_duration']

plt.scatter(x,y)

In [ ]:
### Your code is here
fig = plt.figure()

fig.set_size_inches(16, 3)

ax_1 = fig.add_subplot(1, 2, 1)
x = processed_data.groupby('day_of_week').count()['vendor_id']
plt.bar(x.index, x)
plt.title('Количество поездок по дням недели', fontsize = 7)


ax_2 = fig.add_subplot(1, 2, 2)
x = np.exp(np.array(processed_data.groupby('day_of_week').mean()['log_trip_duration']) )
plt.bar([i for i in range(7)], x)

plt.title('Средняя продолжительность(логарифм) поездок по дням недели', fontsize = 7)



In [ ]:
x = processed_data.groupby(['day_of_week', 'hour']).mean()['log_trip_duration']
x

In [ ]:

fig = plt.figure()

fig.set_size_inches(16, 3)

ax_3 = fig.add_subplot(1, 2, 1)
x = processed_data.groupby('hour').count()['vendor_id']
plt.bar(x.index, x)
plt.title('Количество поездок по часам', fontsize = 7)

ax_4 = fig.add_subplot(1, 2, 2)
x = processed_data.groupby('hour').mean()['log_trip_duration']
plt.bar(x.index, x)
plt.title('Средняя продолжительность(логарифм) поездок по часам', fontsize = 7)

plt.show()

In [ ]:
fig, ax = plt.subplots(layout='constrained')


fig.set_size_inches(16, 7)

x = processed_data.groupby(['day_of_week', 'hour']).count()['log_trip_duration']
hours = [i for i in range(24)]


bar_width = 0.1
for i in range(7):
    plt.bar(np.array(hours) + i * bar_width, x[i], width=bar_width, label = f"day of week: {i + 1}")
    


plt.legend()


plt.title('Средняя продолжительность(логарифм) поездок по часам', fontsize = 7)

plt.show()

In [ ]:
### Your code is here
fig = plt.figure()

fig.set_size_inches(16, 3)

ax_1 = fig.add_subplot(1, 2, 1)
x = processed_data.groupby('month').count()['vendor_id']
plt.bar(x.index, x)
plt.title('Количество поездок по месяцам', fontsize = 7)


ax_2 = fig.add_subplot(1, 2, 2)
x = processed_data.groupby('month').mean()['log_trip_duration']
plt.bar(x.index, x)

plt.title('Средняя продолжительность(логарифм) поездок по месяцам', fontsize = 7)



In [ ]:

fig = plt.figure()

fig.set_size_inches(16, 3)

ax_3 = fig.add_subplot(1, 2, 1)
x = processed_data.groupby('month').count()['vendor_id']
plt.bar(x.index, x)
plt.title('Количество поездок по месяцам', fontsize = 7)

ax_4 = fig.add_subplot(1, 2, 2)
x = processed_data.groupby('month').mean()['log_trip_duration']
plt.bar(x.index, x)
plt.title('Средняя продолжительность(логарифм) поездок по месяцам', fontsize = 7)

plt.show()

In [ ]:
trip_dur_dict = {}

for day_of_week in processed_data['day_of_week'].unique():
    
    trip_dur_dict[day_of_week] = processed_data[processed_data['day_of_week']==day_of_week]["log_trip_duration"]


In [ ]:

fig = plt.figure()
fig.set_size_inches(16, 10)

plt.boxplot(trip_dur_dict.values(), labels=trip_dur_dict.keys())


plt.show()


In [ ]:
x = processed_data.groupby(['day_of_week', 'month']).mean()['log_trip_duration']
x[0]

In [ ]:
fig, ax = plt.subplots(layout='constrained')


fig.set_size_inches(16, 7)

x = processed_data.groupby(['day_of_week', 'month']).mean()['log_trip_duration']
hours = [i for i in range(6)]


bar_width = 0.1
for i in range(7):
    plt.bar(np.array(hours) + i * bar_width, x[i], width=bar_width, label = f"day of week: {i + 1}")
    
plt.ylim((6, 6.6))

plt.legend()


plt.title('Средняя продолжительность(логарифм) поездок по часам', fontsize = 7)

plt.show()

Кажется, что полученные выводы из графиков выше достаточны для того, чтобы посчитать признаки, связанные с временем/датой поездки, достаточно важными. На их основе выделим еще пару.

Добавьте следующие бинарные признаки:


1. Была ли в день поездки буря (основываясь на единственном обнаруженном таком дне)
2. Является ли время поездки статистически самым проблемным, то есть час пиком?

Для создания 2ой фичи используйте следующую логику: посчитаем для каждой пары "день недели"-"время суток" среднее значение таргета. Найдем топ-10 самых "больших" пар. Если поездка была совершена во входящее в этот топ время, то ставим 1. Иначе - 0. Получается бинарный признак.

P.S. назовите колонки **anomaly** и **traffic_jam**

In [1]:
processed_data.head()

NameError: name 'processed_data' is not defined

In [ ]:
### Создадим первый бинарный признак
### Your code is here
processed_data['date'] = pd.to_datetime(processed_data['date'])
processed_data.loc[processed_data['date'] == '2016-01-23','anomaly'] = 1
processed_data.loc[processed_data['date'] != '2016-01-23', 'anomaly'] = 0
processed_data['anomaly'] = processed_data['anomaly'].astype(int)
processed_data['anomaly'].value_counts()

In [13]:
### Создадим второй бинарный признак
### Your code is here

data = processed_data.groupby(['day_of_week','hour'])['log_trip_duration'].mean().reset_index()

pick_time_list = data.sort_values('log_trip_duration',ascending=False)[:10]

processed_data['traffic_jam'] = 0
processed_data.loc[
    (processed_data['hour'].isin(pick_time_list['hour'])) & 
    (processed_data['day_of_week'].isin(pick_time_list['day_of_week'])), 
    'traffic_jam'] = 1


Теперь колонки **pickup_datetime**, **date** можно убрать. А про оставшиеся **day_of_week**, **hour**, **month** необходимо подумать:

- С одной стороны, первые две можно убрать, так как на их основе была создана колонка **traffic_jam**
- С другой стороны, зависимость с колонкой **traffic_jam** нелинейная, поэтому можно попробовать использовать все фичи в комбинации. Конечно, лучше попробовать оба варианта и проэкспериментировать, но ради упрощения - оставим все колонки. Хоть **day_of_week**, **hour** и описываются числами, мы понимаем, что это скорее категориальные фичи. Потому что, например, отношение между 23:00 и 00:00 не такое же, как между числами 23 и 0. Закодируем их с помощью OneHotEncoder. 
- Последняя (**month**) очевидно категориальная. Можно закодировать ее тоже через OneHotEncoder.

In [14]:
processed_data = processed_data.drop(['pickup_datetime', 'date'], axis=1)

In [15]:
# tmp_data = processed_data.copy()

In [18]:


processed_data.to_csv('tmp_before_ohe.csv', sep=',')


In [2]:
import pandas as pd
processed_data = pd.read_csv('tmp_before_ohe.csv', sep=',', index_col='id')

In [3]:
### Делаем OneHotEncoding и конкатим с processed_data
# import pandas as pd

# for col in ['day_of_week', 'hour', 'month']:
   
#     ### Your code is here
#     encoded_columns = pd.get_dummies(processed_data[col], prefix=col)
#     encoded_columns = encoded_columns.astype(int)
#     processed_data = pd.concat((processed_data,encoded_columns),axis=1)


from sklearn.preprocessing import OneHotEncoder
import sklearn as skl
# def custom_combiner(feature, category):
#     return str(feature) + "_" + type(category).__name__ + "_" + str(category)

ohe = OneHotEncoder(drop='first')
ohe_array = ohe.fit_transform(processed_data[['day_of_week', 'hour', 'month']]).toarray()



In [4]:
ohe_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.]])

In [5]:
ohe.categories_

[array([0, 1, 2, 3, 4, 5, 6]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([1, 2, 3, 4, 5, 6])]

In [6]:
ohe.categories_
feature_labels = []
count = 0
for i in ohe.categories_:
    if count == 0:
        phrase = 'day_of_week'
    if count == 1:
        phrase = 'hour'
    if count == 2:
        phrase = 'month'
    for j in i:
        if j != 0 and count in [0, 1]:
            feature_labels.append(f"{phrase}_{j}")
        if count == 2 and j != 1:
            feature_labels.append(f"{phrase}_{j}")
    count += 1
    
print(feature_labels)

['day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6']


In [7]:
# лучше потом весь датафрейм перевести в int
# ohe_array_int = []
# for i in ohe_array:
#     ohe_array_int.append(list(map(int, i)))
# ohe_array_int

In [8]:
feature_ohe = pd.DataFrame(ohe_array, columns=feature_labels, index=processed_data.index)

In [9]:
feature_ohe.shape
print(processed_data.shape, feature_ohe.shape)



(1458644, 10) (1458644, 34)


In [10]:
processed_data_ohe = pd.concat([processed_data, feature_ohe], axis=1)

In [11]:
# processed_data = processed_data.drop(['day_of_week_0','hour_0','month_1'], axis =1)
processed_data_ohe.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,day_of_week,hour,month,anomaly,traffic_jam,...,hour_19,hour_20,hour_21,hour_22,hour_23,month_2,month_3,month_4,month_5,month_6
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,0,17,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
id2377394,0,930.399753,0,1.807119,6.498282,6,0,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
id3858529,1,930.399753,0,6.392080,7.661527,1,11,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id3504673,1,930.399753,0,1.487155,6.063785,2,19,4,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
id2181028,1,930.399753,0,1.189925,6.077642,5,13,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
processed_data_ohe = tmp
processed_data_ohe.head()

NameError: name 'tmp' is not defined

In [ ]:
processed_data_ohe.columns.values.tolist()[10:]


In [13]:
# tmp = processed_data_ohe.copy()

for j in processed_data_ohe.columns.values.tolist()[10:]:
    processed_data_ohe[j] = processed_data_ohe[j].astype(int)

In [14]:
pd.set_option('display.max_columns', None)
processed_data_ohe

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,day_of_week,hour,month,anomaly,traffic_jam,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,month_2,month_3,month_4,month_5,month_6
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,0,17,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
id2377394,0,930.399753,0,1.807119,6.498282,6,0,6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
id3858529,1,930.399753,0,6.392080,7.661527,1,11,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id3504673,1,930.399753,0,1.487155,6.063785,2,19,4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
id2181028,1,930.399753,0,1.189925,6.077642,5,13,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id2376096,1,1053.529749,0,1.226296,6.658011,4,13,4,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
id1049543,0,930.399753,0,6.056935,6.486161,6,7,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id2304944,1,930.399753,0,7.832952,6.639876,4,6,4,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [15]:
processed_data_ohe = processed_data_ohe.drop(['day_of_week','hour','month'], axis =1)


In [16]:
pd.set_option('display.max_columns', None)
processed_data_ohe.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,anomaly,traffic_jam,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,month_2,month_3,month_4,month_5,month_6
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
id2377394,0,930.399753,0,1.807119,6.498282,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
id3858529,1,930.399753,0,6.392080,7.661527,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id3504673,1,930.399753,0,1.487155,6.063785,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
id2181028,1,930.399753,0,1.189925,6.077642,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [17]:
processed_data_ohe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1458644 entries, id2875421 to id1209952
Data columns (total 41 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   vendor_id           1458644 non-null  int64  
 1   passenger_count     1458644 non-null  float64
 2   store_and_fwd_flag  1458644 non-null  int64  
 3   distance_km         1458644 non-null  float64
 4   log_trip_duration   1458644 non-null  float64
 5   anomaly             1458644 non-null  int64  
 6   traffic_jam         1458644 non-null  int64  
 7   day_of_week_1       1458644 non-null  int64  
 8   day_of_week_2       1458644 non-null  int64  
 9   day_of_week_3       1458644 non-null  int64  
 10  day_of_week_4       1458644 non-null  int64  
 11  day_of_week_5       1458644 non-null  int64  
 12  day_of_week_6       1458644 non-null  int64  
 13  hour_1              1458644 non-null  int64  
 14  hour_2              1458644 non-null  int64  
 15  hour_3    

In [18]:
processed_data_ohe[:5].to_csv('HW_7_res.csv', sep=',')

Что же, мы с Вами научились с помощью EDA визуализаций понимать важность признаков не только постфактум, но и до того, как сформировался финальный датасет - на этапе **выделения базовых фичей**.

Обратимся теперь к методам фильтрации - например, применим корреляционный анализ для одной пары фичей, чтобы понять, нет ли относительно этих колонок в нашем датасете избытка информации. 

Вспомним так же изначальные вещественные признаки - distance_km, passenger_count

Посчитайте корреляцию между ними. Есть ли какие-то основания для беспокойства?

In [24]:
### Your code is here

first = processed_data_ohe["distance_km"] 
second = processed_data_ohe["passenger_count"]
df_for_corr = pd.merge(first, second, right_index = True,
               left_index = True)
# df_for_corr = pd.merge(df_for_corr, processed_data_ohe["log_trip_duration"], right_index = True,
#                left_index = True)

In [26]:
df_for_corr.head()

df_for_corr.corr()

,distance_km,passenger_count
distance_km,1.000000,0.016596
passenger_count,0.016596,1.000000


In [28]:
### На данный момент у нас 41 фича
### Представим, что хочется сократить их количество до 5.
### Воспользуемся для этим каким-нибудь методом обертки
### Например, метод прямого отбора

### Your code is here
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
y = processed_data_ohe["log_trip_duration"]
X = processed_data_ohe.drop('log_trip_duration', axis=1)
linreg = LinearRegression()

sfs = SequentialFeatureSelector(linreg, n_features_to_select=5)
sfs.fit(X, y)
sfs.get_support()
sfs.transform(X).shape


(1458644, 5)

In [29]:
### Перечислите фичи, которые окажутся наиболее желанными
### Your code is here
sfs.get_feature_names_out()

array(['distance_km', 'traffic_jam', 'day_of_week_6', 'hour_5', 'hour_6'],
      dtype=object)

In [32]:
### Представим, что мы с Вами ничего не знаем про Кросс-Валидацию и отложенную выборку
### Замерьте качество линейной регрессии на данных фичах
### Сильно ли оно отличается от полученного на Кросс-Валидации в прошлом уроке в ДЗ?

### Your code is here
processed_data_new = X[sfs.get_feature_names_out()]
linreg.fit(processed_data_new,y)
error = np.mean((linreg.predict(processed_data_new) - y) ** 2)
round(error,3)


0.414